In [1]:
import cv2
import numpy as np
import os
import pandas as pd
import math
import matplotlib.pyplot as plt
from os.path import isfile, join
from PIL import Image


In [2]:
pathIn= './good_sequence1/'
pathOut = 'seq1_raw.mp4'

# image1 = Image.open(pathIn+'gs00001.jpg')
# image1.show()

In [3]:
# Read in labels and extract angles vector
# good_sequence1, 40 frames long, from:
# center/1479424375308559998.jpg
# good_sequence2, 160 frames long, from:
# center/1479424398062555784.jpg

table = pd.read_csv('labels.txt',delimiter=' ',names = ["filename", "steer", "torque", "velocity"])
# print(table) #33808 rows x 4 columns

seq1_cond = table['filename'] == 'center/1479424375308559998.jpg'
seq2_cond = table['filename'] == 'center/1479424398062555784.jpg'

seq1_idx = table.index[seq1_cond].tolist()
seq2_idx = table.index[seq2_cond].tolist()
print(seq1_idx)
print(seq2_idx)

[3188]
[3643]


In [4]:
true_angle1 = []
true_angle2 = []
for i in range(40): #extracts steering angles for sequence 1
    idx = seq1_idx[0] + i
    true_angle1.append(table.loc[idx,'steer'])
    
for i in range(160): #extracts steering angles for sequence 1
    idx = seq2_idx[0] + i
    true_angle2.append(table.loc[idx,'steer'])

print(len(true_angle1))
print(len(true_angle2))
# true_angle1 = np.asarray(true_angle1)
# print(true_angle1[1])

40
160


In [38]:
def plot_ray(point, tru_angle, length,exaggerate=False):  
    if exaggerate is True:
        tru_angle=tru_angle*3 
#         pred_angle=pred_angle*2
    startx, starty = point
    endy1 = starty + length * math.sin(math.radians(90.0)+tru_angle)  #default is positive-x axis,
    endx1 = startx + length * math.cos(math.radians(90.0)+tru_angle)  #we need common driving angle
    
#     endy2 = starty + length * math.sin(math.radians(90.0)+pred_angle)  #default is positive-x axis,
#     endx2 = startx + length * math.cos(math.radians(90.0)+pred_angle)
    
#     fig, (ax1, ax2) = plt.subplots(1, 2)
    fig, ax1 = plt.subplots(1, 1)
    fig.suptitle('True steering angle (blue), predicted steering angle(red)')
    ax1.plot([startx, endx1], [starty, endy1],'b')
#     ax2.plot([startx, endx2], [starty, endy2],'r')
    ax1.set_ylim([-1, 1])
    ax1.set_xlim([-1, 1])
#     ax2.set_ylim([-1, 1])
#     ax2.set_xlim([-1, 1])
    fig.show()
    return fig

png_PATH = './png_folder/'

for i in range(40):
    fig = plot_ray([0,0],true_angle1[i],1.0, exaggerate=True)
    fig.savefig(png_PATH+'wheel_1_'+str(i)+'.png')


In [41]:
pathIn='./png_folder/'
pathOut='angle_demo.mp4'

fps = 5 #5 pics per sec
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
print(files[0],files[1],files[2],files[3])

# Filenames in sequence are renamed for making video
# #for sorting the file names properly
files.sort(key = lambda x: x[5:-4])
files.sort()
print(files,'after sort')

wheel100039.png wheel100005.png wheel100011.png wheel100010.png
['wheel100001.png', 'wheel100002.png', 'wheel100003.png', 'wheel100004.png', 'wheel100005.png', 'wheel100006.png', 'wheel100007.png', 'wheel100008.png', 'wheel100009.png', 'wheel100010.png', 'wheel100011.png', 'wheel100012.png', 'wheel100013.png', 'wheel100014.png', 'wheel100015.png', 'wheel100016.png', 'wheel100017.png', 'wheel100018.png', 'wheel100019.png', 'wheel100020.png', 'wheel100021.png', 'wheel100022.png', 'wheel100023.png', 'wheel100024.png', 'wheel100025.png', 'wheel100026.png', 'wheel100027.png', 'wheel100028.png', 'wheel100029.png', 'wheel100030.png', 'wheel100031.png', 'wheel100032.png', 'wheel100033.png', 'wheel100034.png', 'wheel100035.png', 'wheel100036.png', 'wheel100037.png', 'wheel100038.png', 'wheel100039.png', 'wheel100040.png'] after sort


In [42]:
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
#for sorting the file names properly
files.sort(key = lambda x: x[5:-4])
for i in range(len(files)):
    filename=pathIn + files[i]
    #reading each files
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)

for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()